In [11]:
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, Flatten, MaxPool2D, Dense, Dropout, GlobalAveragePooling2D, Reshape, LSTM, Input
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import numpy as np
from skimage import transform
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import seaborn as sns
import splitfolders

In [12]:
splitfolders.ratio(
    "C://Users//HP//Desktop//kdp//CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
    output="C://Users//HP//Desktop//kdp//dataset",
    seed=8,
    ratio=(0.75, 0.12, 0.13)
)

ValueError: The provided input folder "C://Users//HP//Desktop//kdp//CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone" does not exists.

In [13]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

train_dataset = train_datagen.flow_from_directory('C:/Users/HP/Desktop/kdp/dataset/train',
                                                  target_size=(200, 200),
                                                  color_mode='grayscale', 
                                                  class_mode='categorical', 
                                                  batch_size=100,
                                                  )

test_dataset = test_datagen.flow_from_directory('C:/Users/HP/Desktop/kdp/dataset/test',
                                                target_size=(200, 200),
                                                class_mode='categorical',
                                                color_mode='grayscale',
                                                batch_size=100,
                                                shuffle=False
                                                )

valid_dataset = valid_datagen.flow_from_directory('C:/Users/HP/Desktop/kdp/dataset/val',
                                                  target_size=(200, 200),
                                                  class_mode='categorical',
                                                  batch_size=100,
                                                  color_mode='grayscale',
                                                  )


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:/Users/HP/Desktop/kdp/dataset/train'

In [ ]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=train_dataset.image_shape))
model.add(MaxPool2D(2))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPool2D(2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPool2D(2))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.38))

model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPool2D(2))
model.add(Dropout(0.38))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.summary()

In [ ]:
import keras
METRICS = [
        'accuracy',
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
    
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)

# In[7]:

Info = model.fit(
                 train_dataset,
                 validation_data=valid_dataset,
                 epochs=5,
                 )

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(Info.history[met])
    ax[i].plot(Info.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])


In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
diseases_labels = []

for key, value in train_dataset.class_indices.items():
   diseases_labels.append(key)


In [ ]:

def evaluate(actual, predictions):
    pre = []
    for i in predictions:
        pre.append(np.argmax(i))

    accuracy = (pre == actual).sum() / actual.shape[0]
    print(f'Accuracy: {accuracy * 100:.2f}%')

    precision, recall, f1_score, _ = precision_recall_fscore_support(actual, pre, average='macro')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1_score: {f1_score}')

    fig, ax = plt.subplots(figsize=(10,10))
    conf_mat = confusion_matrix(actual, pre)
    sns.heatmap(conf_mat, annot=True, fmt='.0f', cmap="YlGnBu", xticklabels=diseases_labels, yticklabels=diseases_labels).set_title('Confusion Matrix Heat map')
    plt.show()
    
    if accuracy < 0.50:
        raise ValueError("Model accuracy is less than 60%!")

In [ ]:
evaluate(test_dataset.classes, predictions)

In [ ]:
model.evaluate(test_dataset)

In [ ]:
model.save('model.hdf5')

# LSTM

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)


In [ ]:
Info = model.fit(
                 train_dataset,
                 validation_data=valid_dataset,
                 epochs=5,
                 )


In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

In [ ]:
for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(Info.history[met])
    ax[i].plot(Info.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:

predictions = model.predict(test_dataset)

In [ ]:
diseases_labels = []

for key, value in train_dataset.class_indices.items():
   diseases_labels.append(key)

In [ ]:

evaluate(test_dataset.classes, predictions)

In [ ]:

model.save('modellstm.hdf5')

# Resnet

In [3]:
base_model = ResNet50(weights=None, include_top=False, input_shape=(200,200,1))

In [4]:
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

In [5]:
model = Model(inputs=base_model.input, outputs=predictions)

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=METRICS)

NameError: name 'METRICS' is not defined

In [ ]:

Info = model.fit(
    train_dataset,
    epochs=5,
    validation_data=valid_dataset)

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(20, 3))
ax = ax.ravel()

for i, met in enumerate(['precision', 'recall', 'accuracy', 'loss']):
    ax[i].plot(Info.history[met])
    ax[i].plot(Info.history['val_' + met])
    ax[i].set_title('Model {}'.format(met))
    ax[i].set_xlabel('epochs')
    ax[i].set_ylabel(met)
    ax[i].legend(['train', 'val'])

In [ ]:

predictions = model.predict(test_dataset)

In [ ]:
diseases_labels = []

for key, value in train_dataset.class_indices.items():
   diseases_labels.append(key)

In [ ]:
evaluate(test_dataset.classes, predictions)

In [ ]:
model.save('modelresnet.hdf5')